# Other utility functions

This Jupyter notebook contains reusable helper functions for beam deflection analysis.

The implemented functions are used to compute:
- velocity
- acceleration
- bending moment
  
Only applicable for Euler-Bernoulli beam theory

These functions are intended to be **copied or imported** into other notebooks where they are required.

In [ ]:
#Velocity (first derivative of deflection with respect to time )

def velocity(deflection, time):
    """
    Computes velocity v = dw/dt for each point (x,t)

    Parameters
    ----------
    u : torch.Tensor
        Neural network output w(x,t)
    t : torch.Tensor
        Time tensor corresponding to u, requires_grad=True
    x : torch.Tensor
        Position tensor corresponding to u, shape (num_points, 1)

    Returns
    -------
    v : torch.Tensor
        Velocity dw/dt
    x : torch.Tensor
        Returns the input x for convenience in plotting
    """
    v = torch.autograd.grad(
        outputs=u,
        inputs=t,
        grad_outputs=torch.ones_like(u),
        create_graph=True,
        retain_graph=True
    )[0]

    return v, x

In [ ]:
#Acceleration (second derivative of deflection with respect to time )
def acceleration(u, t, x):
    """
    Computes acceleration a = d²w/dt² for each point (x,t)
    
    Parameters
    ----------
    u : torch.Tensor
        Neural network output w(x,t)
    t : torch.Tensor
        Time tensor corresponding to u, requires_grad=True
    x : torch.Tensor
        Position tensor corresponding to u, shape (num_points, 1)

    Returns
    -------
    a : torch.Tensor
        Acceleration d²w/dt², shape (num_points, 1)
    x : torch.Tensor
        Returns the input x for plotting
    """
    # First derivative: velocity
    v = torch.autograd.grad(
        outputs=u,
        inputs=t,
        grad_outputs=torch.ones_like(u),
        create_graph=True,  
        retain_graph=True
    )[0]

    # Second derivative: acceleration
    a = torch.autograd.grad(
        outputs=v,
        inputs=t,
        grad_outputs=torch.ones_like(v),
        create_graph=False, 
        retain_graph=False
    )[0]

    return a, x

In [ ]:
#Bending moment (second derivative of deflection with respect to space multiplied with EI)
def bending_moment(u, x, EI):
    """
    Computes bending moment M(x,t) = -EI * d²w/dx² for each point (x,t)
    
    Parameters
    ----------
    u : torch.Tensor
        Neural network output w(x,t)
    x : torch.Tensor
        Position tensor corresponding to u, shape (num_points, 1), requires_grad=True
    EI : float or torch.Tensor
        Flexural rigidity (E*I)
    
    Returns
    -------
    M : torch.Tensor
        Bending moment, shape (num_points, 1)
    x : torch.Tensor
        Returns input x for plotting
    """
    # First derivative: slope
    slope = torch.autograd.grad(
        outputs=u,
        inputs=x,
        grad_outputs=torch.ones_like(u),
        create_graph=True,  
        retain_graph=True
    )[0]

    # Second derivative: curvature
    curvature = torch.autograd.grad(
        outputs=slope,
        inputs=x,
        grad_outputs=torch.ones_like(slope),
        create_graph=False, 
        retain_graph=False
    )[0]

    # Bending moment
    M = -EI * curvature
    return M, x

In [ ]:
#Animation function
def animate_u_over_time(x, t_list, pred_multi, u_normalizer, u_func, colors, interval=5000):
    """
    Create an animation of exact vs model-predicted u(x,t) over time.

    Args:
        x (np.ndarray): 1D array of x positions, shape [Nx].
        t_list (np.ndarray): 1D array of times, shape [Nt].
        pred_multi (np.ndarray): Predicted outputs, shape [Nt, Nx, C], u in channel 1.
        u_normalizer: normalizer with .denormalize(tensor) for u.
        u_func (callable): exact solution u(x,t).
        colors (list): color list (exact, prediction).
        interval (int): milliseconds between frames.

    Returns:
        ani (FuncAnimation): the animation object.
    """


    from matplotlib.animation import FuncAnimation

    Nt, Nx, C = pred_multi.shape
    assert C >= 2, "Expected u in channel 1."

    # --- Denormalize predictions ---
    u_pred_norm = pred_multi[:, :, 1]                         # shape [Nt, Nx]
    u_pred = u_normalizer.denormalize(
        torch.tensor(u_pred_norm, dtype=torch.float32)
    ).numpy()                                                # shape [Nt, Nx]

    # --- Setup figure ---
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.set_xlabel("x")
    ax.set_ylabel("u(x,t)")
    ax.grid(alpha=0.3)
    ax.set_ylim(-1.5, 1.5)
    # Initial lines
    exact_line, = ax.plot(x, u_func(x, t_list[0]), 
                          label="Exact u", color=colors[0])
    pred_line,  = ax.plot(x, u_pred[0], 
                          label="NN u", linestyle="--", color=colors[1])

    ax.legend()
    ax.set_title(f"t = {t_list[0]:.3f}")

    # --- Update function ---
    def update(frame):
        exact_line.set_ydata(u_func(x, t_list[frame]))
        pred_line.set_ydata(u_pred[frame])
        ax.set_title(f"t = {t_list[frame]:.3f}")
        return exact_line, pred_line

    # --- Build animation ---
    ani = FuncAnimation(fig, update, frames=len(t_list), interval=interval)
    plt.close(fig)  # prevents duplicate static figure in notebooks

    return ani

In [ ]:
#ani = animate_u_over_time(
    #x_test, 
    #t_tests, 
    #pred_test_multi, 
    #u_normalizer, 
    #u, 
    #colors,
    #interval=5000   
#)

#from matplotlib.animation import PillowWriter


#writer = PillowWriter(fps=240)


#writer.frame_duration = 1500  

#ani.save("u_animation_trial.gif", writer=writer

